In [21]:
import os
import requests
import json
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from dotenv import load_dotenv
import time
load_dotenv()
import gaming_companies

In [22]:
# Connect DB Database
cc = os.getenv("credit_card")
token = os.getenv("token")
client = MongoClient("localhost:27017")
client.list_database_names()
db = client["Ironhack"]
db_p_III = client["Project_III"]
c = db.get_collection("companies")

In [23]:
def getCoordinates(address):
    url_geocode = f"https://geocode.xyz/{address}?json=1"
    res = requests.get(url_geocode)
    try:
        return res.json()
    except:
        time.sleep(4)
        getCoordinates(address)

In [24]:
res = getCoordinates("San Francisco")
res

{'standard': {'addresst': {},
  'statename': {},
  'city': 'San Francisco',
  'prov': 'US',
  'countryname': 'United States of America',
  'postal': {},
  'confidence': '0.90'},
 'longt': '-122.40507',
 'alt': {'loc': {'longt': '-122.426263377796',
   'prov': 'CA',
   'city': 'SAN FRANCISCO',
   'postal': '94112',
   'score': '24905',
   'latt': '37.6874300564143'}},
 'elevation': {},
 'latt': '37.75672'}

In [25]:
lat, lon = res["alt"]["loc"]["latt"], res["alt"]["loc"]["longt"]
map_SF = Map(location = [lat, lon], zoom_start = 12)
map_SF

In [26]:
# Geocoding: Converting a place name / address into geographic coordinates

def url_geocode(where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"
    response = requests.get(url_geocode)
    return response.json()

In [27]:
#Create a connection to Foursquare API in order to find out about what we have around a given radius
def requests_4sq(query, lat, lon, radius=None, sort='DISTANCE', limit=1):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&sort={sort}&limit={limit}"
    if radius:
        url += f'&radius={radius}'
    headers = { "accept": "application/json", "Authorization": token}
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [28]:
def upload_collection(c_name, list_):
    db = client.get_database("Project_III")
    c = db[c_name]
    for item in list_:
        c.insert_one(item)

In [29]:
def request_4sq(venue, lat, lon, radius = 3700, sort_by = "DISTANCE", limit = 20):
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={lat}%2C{lon}&radius={radius}&sort={sort_by}&limit={limit}"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    try:
        return requests.get(url, headers = headers).json()
    except:
        print("Request not found")

In [30]:
# Retrieve the midpoint coordinates for San Francisco
sf_lat, sf_lon = gaming_companies.midpoint_coordinates_sf()
# Retrieve the midpoint coordinates for New York
ny_lat,nylon  = gaming_companies.midpoint_coordinates_ny()
# Retrieve the midpoint coordinates for London
ldn_lat, ldn_lon = gaming_companies.midpoint_coordinates_ldn()


In [31]:
starbucks_request_sf = request_4sq("Starbucks", sf_lat, sf_lon)
starbucks_request_ny = request_4sq("Starbucks", ny_lat, nylon)
starbucks_request_ld = request_4sq("Starbucks", ldn_lat, ldn_lon)

In [32]:
# Assuming starbucks_request_sf is your original data structure
list_of_starbucks = starbucks_request_sf['results']  # This extracts the list of dictionaries
upload_collection(c_name="Starbucks", list_=list_of_starbucks)


TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

In [ ]:
starbucks_request_sf

{'results': [{'fsq_id': '4a1e071af964a520bb7b1fe3',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}}],
   'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
     'name': 'Starbucks'}],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 700,
   'geocodes': {'drop_off': {'latitude': 37.763021, 'longitude': -122.410556},
    'main': {'latitude': 37.762898, 'longitude': -122.410549},
    'roof': {'latitude': 37.762898, 'longitude': -122.410549}},
   'link': '/v3/places/4a1e071af964a520bb7b1fe3',
   'location': {'address': '2727 Mariposa St',
    'census_block': '060750228011013',
    'country': 'US',
    'cross_street': 'at Bryant St',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': '2727 Mariposa St (at Bryant St), San Francisco, CA 94110',
    'locality': 'San Francisco',